In [ ]:
!! pip install torch

['Requirement already satisfied: torch in /usr/local/lib/python3.10/dist-packages (2.5.1+cu121)',
 'Requirement already satisfied: filelock in /usr/local/lib/python3.10/dist-packages (from torch) (3.16.1)',
 'Requirement already satisfied: typing-extensions>=4.8.0 in /usr/local/lib/python3.10/dist-packages (from torch) (4.12.2)',
 'Requirement already satisfied: networkx in /usr/local/lib/python3.10/dist-packages (from torch) (3.4.2)',
 'Requirement already satisfied: jinja2 in /usr/local/lib/python3.10/dist-packages (from torch) (3.1.4)',
 'Requirement already satisfied: fsspec in /usr/local/lib/python3.10/dist-packages (from torch) (2024.10.0)',
 'Requirement already satisfied: sympy==1.13.1 in /usr/local/lib/python3.10/dist-packages (from torch) (1.13.1)',
 'Requirement already satisfied: mpmath<1.4,>=1.1.0 in /usr/local/lib/python3.10/dist-packages (from sympy==1.13.1->torch) (1.3.0)',
 'Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.10/dist-packages (from

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim

df = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')

from sklearn.model_selection import train_test_split

# Assume your DataFrame is named 'df' and has a column 'label' for class labels

# Stratified sampling to retain class balance
df_sampled, _ = train_test_split(df, test_size=0.70, stratify=df['label'], random_state=42)

df = df_sampled

############################################################
#####   do this again for a smaller working dataset   ######
############################################################
df_sampled, _ = train_test_split(df, test_size=0.20, stratify=df['label'], random_state=42)


df = df_sampled

# Create a mapping from string labels to numerical indices
class_mapping = {'dog': 0, 'cat': 1, 'bird': 2} #Example mapping, adjust if needed
#Reverse mapping for later use
reverse_mapping = {0: 'dog', 1: 'cat', 2: 'bird'}
#Map the string labels to numerical indices in the dataframe
df['label'] = df['label'].map(class_mapping)

from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim

def create_dataloaders(train_df, test_df, root_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = CustomDataset(train_df, os.path.join(root_dir, 'train'), transform=transform)
    test_dataset = CustomDataset(test_df, os.path.join(root_dir, 'train'), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Prepare the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(df['label'].unique())  # Assuming df contains your label data
model = SimpleCNN(num_classes).to(device)

## data loaders
train_loader, test_loader = create_dataloaders(train_df, test_df, root_dir, batch_size)
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 30
# Train the model (make sure to define train_loader and num_epochs)
train_model(model, train_loader, criterion, optimizer, device, num_epochs)



Epoch 1/30, Loss: 4.9589, Accuracy: 34.38%
Epoch 2/30, Loss: 1.0772, Accuracy: 42.71%
Epoch 3/30, Loss: 1.0515, Accuracy: 42.36%
Epoch 4/30, Loss: 1.0075, Accuracy: 46.53%
Epoch 5/30, Loss: 0.9868, Accuracy: 48.61%
Epoch 6/30, Loss: 0.8814, Accuracy: 57.29%
Epoch 7/30, Loss: 0.8129, Accuracy: 65.28%
Epoch 8/30, Loss: 0.7466, Accuracy: 68.06%
Epoch 9/30, Loss: 0.6729, Accuracy: 68.40%
Epoch 10/30, Loss: 0.5640, Accuracy: 77.78%
Epoch 11/30, Loss: 0.4709, Accuracy: 82.64%
Epoch 12/30, Loss: 0.3688, Accuracy: 88.54%
Epoch 13/30, Loss: 0.2649, Accuracy: 90.62%
Epoch 14/30, Loss: 0.2161, Accuracy: 93.40%
Epoch 15/30, Loss: 0.1509, Accuracy: 94.79%
Epoch 16/30, Loss: 0.1143, Accuracy: 97.57%
Epoch 17/30, Loss: 0.0722, Accuracy: 98.96%
Epoch 18/30, Loss: 0.0668, Accuracy: 98.61%
Epoch 19/30, Loss: 0.0597, Accuracy: 98.26%
Epoch 20/30, Loss: 0.0412, Accuracy: 98.96%
Epoch 21/30, Loss: 0.1130, Accuracy: 96.53%
Epoch 22/30, Loss: 0.0619, Accuracy: 98.61%
Epoch 23/30, Loss: 0.0533, Accuracy: 98.6

## Evaluating Base CNN

In [ ]:

def evaluate_model(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    report = classification_report(all_labels, all_preds, target_names=reverse_mapping.values())
    print(report)


print('----------------------------------')
print('Evaluating Train data accuracy')
print('----------------------------------')
evaluate_model(model, train_loader, device)

print('----------------------------------')
print('Evaluating Test data accuracy')
print('----------------------------------')
evaluate_model(model, test_loader, device)

----------------------------------
Evaluating Train data accuracy
----------------------------------
              precision    recall  f1-score   support

         dog       1.00      1.00      1.00        96
         cat       1.00      1.00      1.00        96
        bird       1.00      1.00      1.00        96

    accuracy                           1.00       288
   macro avg       1.00      1.00      1.00       288
weighted avg       1.00      1.00      1.00       288

----------------------------------
Evaluating Test data accuracy
----------------------------------


KeyboardInterrupt: 

## Creating the basic functions for the project
I've decied to use resnet18 for this assignment, completely missed the fact that I had to start with a baseline CNN.

My initial attempts at this revolved aroung using a resnet18 that was not pretrained, and the best result I got was a whopping 54%

Fortunately that wasn't necessary and I could use a pretrained version accoring to whoever I asked. Life-savers.

In [ ]:
import pandas as pd
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import torchvision.transforms as transforms
import torch.nn as nn


# 1. Function to split the dataset
def split_data(df, test_size=0.90, random_state=42):
    return train_test_split(df, test_size=test_size, stratify=df['label'], random_state=random_state)


# 2. Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        while True:
            img_name = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
            try:
                image = Image.open(img_name).convert('RGB')
                label = self.dataframe.iloc[idx, 1]
                label = torch.tensor(label, dtype=torch.long)  # Convert label to tensor

                if self.transform:
                    image = self.transform(image)

                return image, label
            except FileNotFoundError:
                idx = (idx + 1) % len(self.dataframe)  # Wrap around index




# 3. Function to create dataloaders
def create_dataloaders(train_df, test_df, root_dir, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = CustomDataset(train_df, os.path.join(root_dir, 'train'), transform=transform)
    test_dataset = CustomDataset(test_df, os.path.join(root_dir, 'train'), transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


# 4. Function to define and prepare the model
def get_model(num_classes, device):
    model = torchvision.models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, num_classes)  # Adjust final layer
    model = model.to(device)
    return model


def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the gradients from the previous step
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backpropagate the gradients
            optimizer.step()  # Update model parameters

            # Accumulate the loss
            running_loss += loss.item() * images.size(0)

            # Compute accuracy for this batch
            _, preds = torch.max(outputs, 1)  # Get predicted class
            correct_predictions += torch.sum(preds == labels).item()  # Count correct predictions
            total_predictions += labels.size(0)  # Count total predictions

        # Compute the loss and accuracy for the epoch
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_predictions / total_predictions

        # Print the loss and accuracy for the epoch
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy * 100:.2f}%')

    print('Training complete')




## Splitting the data

Due to limited resources, I decided to use a smaller chunk of the data than what was provided. I figured, around 30% of the total should be enough

Then split that again into 80% for training and 20% for testing

I did make sure that the classes did not get imbalanced and hence the hack code below

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')

from sklearn.model_selection import train_test_split

# Assume your DataFrame is named 'df' and has a column 'label' for class labels

# Stratified sampling to retain class balance
df_sampled, _ = train_test_split(df, test_size=0.70, stratify=df['label'], random_state=42)

# Display the sampled DataFrame
print(df_sampled.head())

# Verify the class distribution
print(df_sampled['label'].value_counts(normalize=True))

df = df_sampled

############################################################
#####   do this again for a smaller working dataset   ######
############################################################
df_sampled, _ = train_test_split(df, test_size=0.20, stratify=df['label'], random_state=42)

# Display the sampled DataFrame
print(df_sampled.head())

# Verify the class distribution
print(df_sampled['label'].value_counts(normalize=True))

df = df_sampled

           filename label
3903   cat_9975.png   cat
7692  bird_6839.png  bird
4443  dog_14123.png   dog
9995  bird_2072.png  bird
6974  bird_3200.png  bird
label
cat     0.333333
bird    0.333333
dog     0.333333
Name: proportion, dtype: float64
            filename label
6635    dog_5200.png   dog
11768   bird_112.png  bird
11057    dog_255.png   dog
4358     dog_194.png   dog
11773  cat_12188.png   cat
label
dog     0.333333
bird    0.333333
cat     0.333333
Name: proportion, dtype: float64


## Running the Model

To make sure the data trained was generalisable, I kept the Epoch at 30, which has the best results I believe, they're not very overfit.


In [ ]:
# Create a mapping from string labels to numerical indices
class_mapping = {'dog': 0, 'cat': 1, 'bird': 2}
#Reverse mapping for later use
reverse_mapping = {0: 'dog', 1: 'cat', 2: 'bird'}
#Map the string labels to numerical indices in the dataframe
df['label'] = df['label'].map(class_mapping)

root_dir = '/content/drive/MyDrive/dataset'
num_epochs=30
batch_size=32

# 6. Main function to orchestrate everything
#def main(df, root_dir, num_epochs=10, batch_size=32):
# Split the data
train_df, test_df = split_data(df)

# Create dataloaders
train_loader, test_loader = create_dataloaders(train_df, test_df, root_dir, batch_size)

# Prepare the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(df['label'].unique())
model = get_model(num_classes, device)

# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Train the model
train_model(model, train_loader, criterion, optimizer, device, num_epochs)




/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/30, Loss: 0.9223, Accuracy: 52.08%
Epoch 2/30, Loss: 0.2276, Accuracy: 96.18%
Epoch 3/30, Loss: 0.0649, Accuracy: 100.00%
Epoch 4/30, Loss: 0.0268, Accuracy: 100.00%
Epoch 5/30, Loss: 0.0103, Accuracy: 100.00%
Epoch 6/30, Loss: 0.0095, Accuracy: 100.00%
Epoch 7/30, Loss: 0.0088, Accuracy: 100.00%
Epoch 8/30, Loss: 0.0053, Accuracy: 100.00%
Epoch 9/30, Loss: 0.0035, Accuracy: 100.00%
Epoch 10/30, Loss: 0.0041, Accuracy: 100.00%
Epoch 11/30, Loss: 0.0027, Accuracy: 100.00%
Epoch 12/30, Loss: 0.0025, Accuracy: 100.00%
Epoch 13/30, Loss: 0.0031, Accuracy: 100.00%
Epoch 14/30, Loss: 0.0034, Accuracy: 100.00%
Epoch 15/30, Loss: 0.0023, Accuracy: 100.00%
Epoch 16/30, Loss: 0.0019, Accuracy: 100.00%
Epoch 17/30, Loss: 0.0020, Accuracy: 100.00%
Epoch 18/30, Loss: 0.0038, Accuracy: 100.00%
Epoch 19/30, Loss: 0.0015, Accuracy: 100.00%
Epoch 20/30, Loss: 0.0017, Accuracy: 100.00%
Epoch 21/30, Loss: 0.0014, Accuracy: 100.00%
Epoch 22/30, Loss: 0.0020, Accuracy: 100.00%
Epoch 23/30, Loss: 0.

## Checking accuracy based on test data


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report

# Function to evaluate the model on the test set
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average='macro')  # Macro recall
    f1 = f1_score(all_labels, all_preds, average='macro')  # Macro F1-score
    class_report = classification_report(all_labels, all_preds, target_names=['bird', 'cat', 'dog'])

    return accuracy, recall, f1, class_report



# Evaluate the model
accuracy, recall, f1, class_report = evaluate_model(model, train_loader, device)
print('----------------------------------')
print('Evaluating Train data accuracy')
print('----------------------------------')
# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print('Classification Report:')
print(class_report)

print('----------------------------------')
print('Evaluating Test data accuracy')
print('----------------------------------')
# Evaluate the model
accuracy, recall, f1, class_report = evaluate_model(model, test_loader, device)

# Print the results
print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print('Classification Report:')
print(class_report)

# Optionally, map the numeric labels back to the class names for display
reverse_mapping = {0: 'dog', 1: 'cat', 2: 'bird'}
test_df['label'] = test_df['label'].map(reverse_mapping)
#print(test_df)


----------------------------------
Evaluating Train data accuracy
----------------------------------
Accuracy: 1.0000
Recall: 1.0000
F1-Score: 1.0000
Classification Report:
              precision    recall  f1-score   support

        bird       1.00      1.00      1.00        96
         cat       1.00      1.00      1.00        96
         dog       1.00      1.00      1.00        96

    accuracy                           1.00       288
   macro avg       1.00      1.00      1.00       288
weighted avg       1.00      1.00      1.00       288

----------------------------------
Evaluating Test data accuracy
----------------------------------
Accuracy: 0.8152
Recall: 0.8152
F1-Score: 0.8144
Classification Report:
              precision    recall  f1-score   support

        bird       0.80      0.77      0.79       864
         cat       0.77      0.76      0.76       864
         dog       0.87      0.91      0.89       864

    accuracy                           0.82      2592
  

## Looks good to me

## Now for the main objective the 3000 imges of test

In [ ]:
import os
import pandas as pd
root_dir = '/content/drive/MyDrive/dataset'
# Get the list of image filenames in the 'test' folder
test_folder = rf'{root_dir}/test'
test_filenames = [f for f in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder, f))]

# Create a dataframe with the filenames
test_df = pd.DataFrame(test_filenames, columns=['file_name'])
print(test_df)

         file_name
0     img_2614.png
1        img_3.png
2      img_241.png
3     img_2338.png
4     img_2346.png
...            ...
3015  img_1233.png
3016  img_1084.png
3017  img_1093.png
3018  img_1181.png
3019  img_1059.png

[3020 rows x 1 columns]


In [ ]:
def predict_labels(model, test_df, root_dir, device):
    model.eval()
    predictions = []

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    with torch.no_grad():
        for idx in range(len(test_df)):
            img_name = os.path.join(root_dir, 'test', test_df.iloc[idx, 0])
            try:
                image = Image.open(img_name).convert('RGB')
                image = transform(image).unsqueeze(0).to(device)
                outputs = model(image)
                _, predicted = torch.max(outputs, 1)
                predictions.append(predicted.item())
            except FileNotFoundError:
                predictions.append(None)  # If file is not found, append None

    return predictions

# Run predictions and save them in test_df
test_df['label'] = predict_labels(model, test_df, root_dir, device)
print(test_df)

         file_name  label
0     img_2614.png      1
1        img_3.png      1
2      img_241.png      2
3     img_2338.png      0
4     img_2346.png      0
...            ...    ...
3015  img_1233.png      0
3016  img_1084.png      1
3017  img_1093.png      2
3018  img_1181.png      0
3019  img_1059.png      0

[3020 rows x 2 columns]


In [ ]:
reverse_mapping = {0: 'dog', 1: 'cat', 2: 'bird'}
test_df['label'] = test_df['label'].map(reverse_mapping)
print(test_df)

         file_name label
0     img_2614.png   cat
1        img_3.png   cat
2      img_241.png  bird
3     img_2338.png   dog
4     img_2346.png   dog
...            ...   ...
3015  img_1233.png   dog
3016  img_1084.png   cat
3017  img_1093.png  bird
3018  img_1181.png   dog
3019  img_1059.png   dog

[3020 rows x 2 columns]


In [ ]:
test_df.to_csv('results.csv')